In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import os

def make_dir(path):
    if os.path.isdir(data_path):
        pass
    else:
        os.path.make_dir(path)

In [37]:
# 初期設定
import sagemaker
import os
import boto3
import re
import numpy as np

sagemaker_session = sagemaker.Session()

# AWS設定
role = 'FullAccessHan'
region = boto3.Session().region_name
bucket='sagemaker-han'
prefix = 'sagemaker/cnn-cifar10'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

# Local設定
data_dir = 'cifar10/testing'

In [24]:
# データセットの準備 → S3に保存　
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

train_data = datasets.CIFAR10(root=training_dir, train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root=test_dir, train=False, download=True, transform=transform)

training_data_loader = DataLoader(train_data, batch_size=len(train_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

100.0%


Extracting mnist_png/training/cifar-10-python.tar.gz to mnist_png/training


100.0%


Extracting mnist_png/testing/cifar-10-python.tar.gz to mnist_png/testing
input spec (in this case, just an S3 path): s3://sagemaker-han/sagemaker/cnn-cifar10


In [26]:
# Train Script
!pygmentize ./src/resnet_train.py

import argparse
import json
import logging
import os
import sys
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
# https://colab.research.google.com/github/seyrankhademi/ResNet_CIFAR10/blob/master/CIFAR10_ResNet.ipynb

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    

In [30]:
# Create Training Container
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="resnet_train.py",
                    role=role,
                    source_dir = "src",
                    framework_version='1.12.0',
                    py_version='py38',
                    instance_count=1,
                    instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                    })

In [31]:
estimator.fit({'training': inputs})

2022-08-20 08:30:34 Starting - Starting the training job...
2022-08-20 08:31:00 Starting - Preparing the instances for trainingProfilerReport-1660984232: InProgress
......
2022-08-20 08:32:12 Downloading - Downloading input data......
2022-08-20 08:33:12 Training - Downloading the training image......
2022-08-20 08:34:18 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-20 08:34:15,963 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-20 08:34:15,965 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-20 08:34:15,974 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-20 08:34:15,983 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-20 08:34:16,685 sagemaker-training-toolkit INFO     N

Train Epoch: 1 [12800/50000 (26%)] Loss: -15.599797
INFO:__main__:Train Epoch: 1 [12800/50000 (26%)] Loss: -15.599797
Train Epoch: 1 [25600/50000 (51%)] Loss: -75.351624
INFO:__main__:Train Epoch: 1 [25600/50000 (51%)] Loss: -75.351624
Train Epoch: 1 [38400/50000 (77%)] Loss: -376.315094
INFO:__main__:Train Epoch: 1 [38400/50000 (77%)] Loss: -376.315094
/opt/conda/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Test set: Average loss: -1459.9684, Accuracy: 1851/10000 (19%)
INFO:__main__:Test set: Average loss: -1459.9684, Accuracy: 1851/10000 (19%)
Saving the model.
INFO:__main__:Saving the model.
2022-08-20 08:40:13,274 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2022-08-20 08:40:13,274 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2022-08-20

In [34]:
!pygmentize ./src/resnet_deploy.py

import torch
import os

import sys
import torch
import torch.nn as nn
import torch.nn.functional as F


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
        )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
    

In [54]:
from sagemaker.pytorch.model import PyTorchModel

model_data = 's3://sagemaker-us-west-2-608095525235/pytorch-training-2022-08-20-08-30-28-845/output/model.tar.gz'
inference_inputs = 's3://sagemaker-han/sagemaker/batch_transform'
output_s3_path = 'https://sagemaker-han.s3.us-west-2.amazonaws.com/sagemaker/batch_transform_output_one_predict'

pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='resnet_train.py',
                             source_dir = 'src',
                             framework_version='1.12.0',
                             py_version='py38',
                             role = role)
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

-----!

In [87]:
# Select 5 images and labels from test set
import numpy as np
n_sample = 1100
n_test = len(test_data)
sampled_index = np.random.choice(n_test, size=n_sample, replace=False)
sampled_testimg = np.array([test_data[i][0].numpy() for i in sampled_index])
sampled_testlabel = np.array([test_data[i][1] for i in sampled_index])

# Run inference and pick up the most likely label based on the score
labels = []
for split_sample in np.array_split(sampled_testimg,3):
    prediction_scores = predictor.predict(split_sample)
    labels += np.argmax(prediction_scores,axis = 1).tolist()
    
print("Predicted labels: {}".format(labels))
print("Ground Truth:     {}".format(sampled_testlabel))
predictor.delete_endpoint()

Predicted labels: [8, 6, 6, 8, 8, 8, 6, 6, 8, 8, 6, 8, 6, 8, 6, 6, 6, 6, 8, 6, 8, 6, 8, 6, 6, 8, 6, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 8, 8, 6, 6, 6, 8, 6, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 6, 8, 8, 8, 6, 6, 8, 8, 8, 6, 8, 6, 6, 6, 6, 6, 6, 8, 6, 6, 6, 8, 6, 6, 6, 6, 8, 6, 8, 8, 8, 6, 8, 8, 6, 6, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 8, 6, 6, 6, 8, 6, 8, 6, 6, 6, 6, 8, 8, 8, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 8, 6, 6, 8, 8, 6, 6, 6, 8, 6, 8, 6, 8, 6, 6, 6, 8, 8, 8, 6, 8, 8, 6, 6, 8, 6, 6, 8, 6, 8, 6, 6, 6, 8, 8, 6, 8, 8, 6, 6, 6, 8, 8, 6, 8, 6, 8, 6, 6, 6, 8, 6, 8, 8, 6, 6, 8, 6, 8, 8, 8, 8, 8, 6, 6, 6, 8, 8, 8, 8, 6, 8, 8, 8, 8, 6, 6, 6, 8, 6, 6, 8, 6, 6, 8, 6, 6, 8, 6, 6, 6, 6, 8, 6, 6, 6, 8, 6, 6, 4, 6, 6, 8, 6, 6, 8, 8, 6, 6, 6, 8, 8, 6, 6, 8, 6, 8, 6, 8, 6, 6, 8, 8, 8, 6, 8, 6, 6, 8, 6, 6, 8, 8, 8, 6, 6, 6, 6, 8, 6, 8, 6, 6, 6, 6, 6, 6, 8, 6, 8, 8, 6, 8, 8, 8, 8, 6, 6, 8, 8, 6, 8, 8, 6, 8, 6, 8, 6, 8, 8, 8, 6, 8, 8, 6, 6, 6, 8, 6, 6, 6, 6, 6, 6, 

In [104]:
# sample_dataloaded = test_data_loaded[0][:1100].numpy()
# sampled_1100_path = './data/sampled_1100.npy'
# np.save(sampled_1100_path,sample_dataloaded)

# prefix = 'sagemaker-han/sagemaker/cifar10'

# new_input = sagemaker_session.upload_data(path=sampled_1100_path, bucket=bucket, key_prefix=prefix)

In [160]:
# datset files
dataset_jsonl_file="./data/cifar10.jsonl"

with open(dataset_jsonl_file, "w+") as outfile:
    for row in sample_dataloaded:
        json.dump(row.tolist(), outfile)
        outfile.write('\n')
        
prefix = 'sagemaker-han/sagemaker/cifar10'

new_input = sagemaker_session.upload_data(path=dataset_jsonl_file, bucket=bucket, key_prefix=prefix)

In [165]:
!pip install jsonlines
import jsonlines
import numpy as np
tmp = './data/test_img_path.json'
inputs = []
with jsonlines.open(dataset_jsonl_file) as reader:
    for obj in reader:
        inputs.append(np.array(obj))
inputs.pop(0)
print(len(inputs))
print


TypeError: pop expected at most 1 argument, got 2

In [138]:
model_data = 's3://sagemaker-us-west-2-608095525235/pytorch-training-2022-08-20-08-30-28-845/output/model.tar.gz'
inference_inputs = 's3://sagemaker-han/sagemaker/batch_transform'
output_s3_path = 'https://sagemaker-han.s3.us-west-2.amazonaws.com/sagemaker/batch_transform_output_one_predict'

pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='resnet_deploy.py',
                             source_dir = 'src',
                             framework_version='1.12.0',
                             py_version='py38',
                             role = role)

In [139]:
# Batch Tranform

max_concurrent_transforms = None
max_payload = None

inference_inputs = 's3://sagemaker-han/sagemaker/batch_transform'
output_s3_path = 'https://sagemaker-han.s3.us-west-2.amazonaws.com/sagemaker/batch_transform_output_cifar10_{}_{}'.format(max_concurrent_transforms,max_payload)

transformer = pytorch_model.transformer(instance_count=1, 
                                        instance_type="ml.m4.xlarge",
                                        output_path=output_s3_path,
                                        max_concurrent_transforms = max_concurrent_transforms,
                                        max_payload = max_payload
                                       )


In [140]:

transformer.transform(
    data=new_input,
    wait=True,
    content_type='application/jsonlines',    
    split_type='Line'
)

2022-08-21T17:20:13,468 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-08-21T17:20:13,601 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.6.0
TS Home: /opt/conda/lib/python3.8/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 3088 M
Python executable: /opt/conda/bin/python3.8
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 4
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Limit Maximum Image Pixels: true
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metric


2022-08-21T17:20:20,876 [INFO ] pool-2-thread-5 ACCESS_LOG - /169.254.255.130:33978 "GET /ping HTTP/1.1" 200 35
2022-08-21T17:20:20,878 [INFO ] pool-2-thread-5 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:56a3d289cfe0,timestamp:1661102420
2022-08-21T17:20:20,907 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:33982 "GET /execution-parameters HTTP/1.1" 404 2
2022-08-21T17:20:20,910 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - Requests4XX.Count:1|#Level:Host|#hostname:56a3d289cfe0,timestamp:1661102420
2022-08-21T17:20:20,876 [INFO ] pool-2-thread-5 ACCESS_LOG - /169.254.255.130:33978 "GET /ping HTTP/1.1" 200 35
2022-08-21T17:20:20,878 [INFO ] pool-2-thread-5 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:56a3d289cfe0,timestamp:1661102420
2022-08-21T17:20:20,907 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:33982 "GET /execution-parameters HTTP/1.1" 404 2
2022-08-21T17:20:20,910 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - Requests4XX.Count

2022-08-21T17:20:23.065:[sagemaker logs]: sagemaker-han/sagemaker-han/sagemaker/cifar10/cifar10.jsonl:     raise JSONDecodeError("Extra data", s, end)
2022-08-21T17:20:23.065:[sagemaker logs]: sagemaker-han/sagemaker-han/sagemaker/cifar10/cifar10.jsonl: json.decoder.JSONDecodeError: Extra data: line 2 column 1 (char 65293)


UnexpectedStatusException: Error for Transform job pytorch-inference-2022-08-21-17-14-31-810: Failed. Reason: AlgorithmError: See job logs for more information

In [131]:
transformer.transform(
    data=new_input,
    wait=True,
    content_type='application/jsonlines',    
    split_type='Line'
)

2022-08-21T16:49:47,004 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-08-21T16:49:47,157 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.6.0
TS Home: /opt/conda/lib/python3.8/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 3070 M
Python executable: /opt/conda/bin/python3.8
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 4
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Limit Maximum Image Pixels: true
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metric

2022-08-21T16:49:53,855 [INFO ] pool-2-thread-5 ACCESS_LOG - /169.254.255.130:35634 "GET /ping HTTP/1.1" 200 34
2022-08-21T16:49:53,856 [INFO ] pool-2-thread-5 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:92efc52e3711,timestamp:1661100593
2022-08-21T16:49:53,900 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:35648 "GET /execution-parameters HTTP/1.1" 404 2
2022-08-21T16:49:53,903 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - Requests4XX.Count:1|#Level:Host|#hostname:92efc52e3711,timestamp:1661100593
2022-08-21T16:49:53,855 [INFO ] pool-2-thread-5 ACCESS_LOG - /169.254.255.130:35634 "GET /ping HTTP/1.1" 200 34
2022-08-21T16:49:53,856 [INFO ] pool-2-thread-5 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:92efc52e3711,timestamp:1661100593
2022-08-21T16:49:53,900 [INFO ] epollEventLoopGroup-3-2 ACCESS_LOG - /169.254.255.130:35648 "GET /execution-parameters HTTP/1.1" 404 2
2022-08-21T16:49:53,903 [INFO ] epollEventLoopGroup-3-2 TS_METRICS - Requests4XX.Count:

2022-08-21T16:49:57,475 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-21T16:49:57,477 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]158
2022-08-21T16:49:57,477 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-21T16:49:57,478 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
2022-08-21T16:49:57,478 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-21T16:49:57,480 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9000.
2022-08-21T16:49:57,475 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-21T16:49:57,477 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]158
2022-08-21T16:49:57,477 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-21T16:49:57,478 [INFO ] W-9000-model_1.0-stdout MODEL_

UnexpectedStatusException: Error for Transform job pytorch-inference-2022-08-21-16-43-47-224: Failed. Reason: AlgorithmError: See job logs for more information